Visualizacion de numero de columnas, filas y muestra de datos 

In [3]:
import pandas as pd

df_data = pd.read_csv('./SI_DATA_POBREZA.csv')

# manejo de valores nulos

df_clean = df_data.dropna()

# numero de filas y columnas

row,col = df_clean.shape;
print('Numero de columnas: ',col,' Numero de filas: ',row)

#impresion de las primeras 11 filas

df_data.head(11)

Numero de columnas:  15  Numero de filas:  286


,AÑO,DEPARTAMENTO,POBLACION TOTAL,POBLACION POBRE EXTREMO,POBLACION POBRE NO EXTREMO,POBLACION NO POBRE,PORCENTAJE_POBREZA,PORCENTAJE POBREZA EXTREMA,CASOS REPORTADOS DE VIOLENCIA,RATIO_VIOLENCIA,PROMEDIO DE AÑOS DE ESTUDIO,RATIO_DE_AÑOS_DE_ESTUDIO_PROMEDIO,INGRESO REAL PROMEDIO PER CAPITA MENSUAL,RATIO_DE_INGRESO_PER_CAPITA,PORCENTAJE_DE_ACCESO_A_SERVICIOS_BASICOS
0,2012,AMAZONAS,"436,165","63,148","130,853","242,164",0.445,0.145,666,0.00153,8.3,0.755,774,1.147,0.500
1,2013,AMAZONAS,"437,480","69,125","137,863","230,492",0.473,0.158,844,0.00193,8.3,0.755,756,1.008,0.547
2,2014,AMAZONAS,"441,984","62,491","164,295","215,198",0.513,0.141,856,0.00194,8.4,0.764,745,0.993,0.513
3,2015,AMAZONAS,"443,783","46,598","141,065","256,120",0.423,0.105,945,0.00213,8.5,0.773,818,1.091,0.530
4,2016,AMAZONAS,"442,069","50,679","124,866","266,524",0.397,0.115,949,0.00215,8.5,0.773,845,0.994,0.559
5,2017,AMAZONAS,"438,629","41,637","111,358","285,634",0.349,0.095,1200,0.00274,8.7,0.791,857,1.008,0.600
6,2018,AMAZONAS,"437,752","27,083","119,620","291,049",0.335,0.062,1397,0.00319,8.8,0.800,881,0.927,0.577
7,2019,AMAZONAS,"439,625","17,251","116,784","305,590",0.305,0.039,1810,0.00412,9.0,0.818,895,0.942,0.589
8,2020,AMAZONAS,"433,062","33,491","122,925","276,646",0.361,0.077,1454,0.00336,8.9,0.809,834,0.878,0.653
9,2021,AMAZONAS,"434,505","39,157","91,613","303,735",0.301,0.090,1720,0.00396,9.1,0.827,955,1.005,0.656


Comprobamos que los datos estan integros y que el numero de columnas y filas corresponden con los que estan en nuestra bd.